# This notebook is for competion using LSTM model and Same values

* multiprocess to speed up the process
* get train data
* get result data with category

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import multiprocessing as mp
import pandas as pd
import numpy as np
from functools import partial
import random

from helper_functions import *

In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv').sort_values(by=['cfips', 'first_day_of_month']).reset_index().drop('index', axis=1)
results = pd.read_csv('lstm_sv_result_with_category.csv')

In [19]:
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1001_2022-12-01,1001,2022-12-01
2,1001_2023-01-01,1001,2023-01-01
3,1001_2023-02-01,1001,2023-02-01
4,1001_2023-03-01,1001,2023-03-01
...,...,...,...
25075,56045_2023-02-01,56045,2023-02-01
25076,56045_2023-03-01,56045,2023-03-01
25077,56045_2023-04-01,56045,2023-04-01
25078,56045_2023-05-01,56045,2023-05-01


In [4]:
results.head()

,Country,MAPE,SV_MAPE,Active,category
0,1001,0.732748,1.835232,1417,lstm
1,1003,0.747286,5.230218,13401,lstm
2,1005,1.273616,1.101416,239,SV
3,1007,9.315642,3.574082,220,SV
4,1009,0.766072,2.116541,789,lstm


## Muitiprocess to do all of the data

In [4]:
%%time
if __name__ == '__main__':
    HORIZON = 8
    WINDOW_SIZE = 9

    train = pd.read_csv('train.csv')
    IDS = train.cfips.unique()

    window_size = WINDOW_SIZE
    horizon = HORIZON
    epoch = 500
    split_size = 0
    
    cfips_list = []
    forecasts = []

    pool = mp.Pool(16)

    func = partial(get_score_for_competition, train, results, window_size, horizon, epoch, split_size)
    results = pool.map(func, IDS)
    pool.close()
    pool.join()
    
    for i in range(len(results)):
        c = results[i][0]
        cfips_list.append(c)
        forecasts.append(results[i][1])

CPU times: total: 6.36 s
Wall time: 58min 42s


In [21]:
test['microbusiness_density'] = np.array(forecasts).reshape((-1))
submission = test[['row_id', 'microbusiness_density']]

In [22]:
submission.head(10)

,row_id,microbusiness_density
0,1001_2022-11-01,3.335410
1,1001_2022-12-01,3.353883
2,1001_2023-01-01,3.372641
3,1001_2023-02-01,3.378253
4,1001_2023-03-01,3.397793
5,1001_2023-04-01,3.420552
6,1001_2023-05-01,3.435101
7,1001_2023-06-01,3.459288
8,1003_2022-11-01,8.237763
9,1003_2022-12-01,8.221723


In [23]:
submission.to_csv('submission_2.csv', index=False)